In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("../data/synthetic/transactions.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.sort_values(["user_id", "timestamp"], inplace=True)

df.head()


,transaction_id,user_id,merchant_id,amount,channel,location,timestamp,device_change,geo_distance,fraud_type,fraud_strategy_version,is_fraud
1130,21235,1,27,5.15,card,rural,2024-01-03 03:48:02,0,4.25,none,0,0
1792,38447,1,93,37.05,netbanking,semi-urban,2024-01-04 07:10:57,0,6.13,none,0,0
2603,38162,1,125,137.87,card,rural,2024-01-05 18:50:51,0,5.54,none,0,0
2685,46092,1,62,47.32,netbanking,rural,2024-01-05 22:04:41,0,6.77,none,0,0
2901,6890,1,144,93.75,card,semi-urban,2024-01-06 06:48:59,0,5.37,none,0,0


In [4]:
df["time_since_last_txn"] = (
    df.groupby("user_id")["timestamp"]
    .diff()
    .dt.total_seconds()
)

df["time_since_last_txn"].fillna(df["time_since_last_txn"].median(), inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\4064972200.py:7: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df["time_since_last_txn"].fillna(df["time_since_last_txn"].median(), inplace=True)


1130     105509.5
1792      98575.0
2603     128394.0
2685      11630.0
2901      31458.0
           ...   
41237    452285.0
42682    221884.0
45146    385930.0
46851    257689.0
49946    482986.0
Name: time_since_last_txn, Length: 50000, dtype: float64

In [5]:
df["txn_count_1h"] = (
    df.groupby("user_id")
      .rolling("1h", on="timestamp")
      .count()["amount"]
      .reset_index(level=0, drop=True)
)


In [6]:
df["txn_count_1h"].fillna(1, inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\1986434824.py:1: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df["txn_count_1h"].fillna(1, inplace=True)


1130     1.0
1792     1.0
2603     1.0
2685     1.0
2901     1.0
        ... 
41237    1.0
42682    1.0
45146    1.0
46851    1.0
49946    1.0
Name: txn_count_1h, Length: 50000, dtype: float64

In [7]:
df["avg_amount_7d"] = (
    df.groupby("user_id", group_keys=False)
      .apply(lambda x: x.sort_values("timestamp")
                        .rolling("7d", on="timestamp")["amount"]
                        .mean())
)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2046175349.py:4: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .rolling("7d", on="timestamp")["amount"]
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2046175349.py:3: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .apply(lambda x: x.sort_values("timestamp")


In [8]:
df["std_amount_7d"] = (
    df.groupby("user_id", group_keys=False)
      .apply(lambda x: x.sort_values("timestamp")
                        .rolling("7d", on="timestamp")["amount"]
                        .std())
)

df["std_amount_7d"].fillna(0, inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\284200328.py:4: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .rolling("7d", on="timestamp")["amount"]
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\284200328.py:3: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .apply(lambda x: x.sort_values("timestamp")
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\284200328.py:8: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid

1130       0.000000
1792      22.556706
2603      69.278280
2685      56.920972
2901      51.984197
            ...    
41237     40.191949
42682    145.034672
45146      1.732412
46851    112.203704
49946     96.385725
Name: std_amount_7d, Length: 50000, dtype: float64

In [9]:
df["amount_zscore"] = (
    (df["amount"] - df["avg_amount_7d"]) /
    (df["std_amount_7d"] + 1)
)

df["amount_zscore"].fillna(0, inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2681095173.py:6: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df["amount_zscore"].fillna(0, inplace=True)


1130     0.000000
1792     0.677090
2603     1.107692
2685    -0.164491
2901     0.557185
           ...   
41237    0.689941
42682   -0.702265
45146    0.448322
46851    0.700860
49946   -0.699846
Name: amount_zscore, Length: 50000, dtype: float64

In [10]:
df["device_change_rolling"] = (
    df.groupby("user_id", group_keys=False)
      .apply(
          lambda x: x.sort_values("timestamp")
                        .rolling("3d", on="timestamp")["device_change"]
                        .mean()
      )
)

df["device_change_rolling"].fillna(0, inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2654625537.py:5: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .rolling("3d", on="timestamp")["device_change"]
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2654625537.py:4: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  lambda x: x.sort_values("timestamp")
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2654625537.py:10: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to a

1130     0.0
1792     0.0
2603     0.0
2685     0.0
2901     0.0
        ... 
41237    0.0
42682    0.0
45146    0.0
46851    0.0
49946    0.0
Name: device_change_rolling, Length: 50000, dtype: float64

In [11]:
df["geo_distance_rolling"] = (
    df.groupby("user_id", group_keys=False)
      .apply(
          lambda x: x.sort_values("timestamp")
                        .rolling("3d", on="timestamp")["geo_distance"]
                        .mean()
      )
)

df["geo_distance_rolling"].fillna(
    df["geo_distance_rolling"].median(), inplace=True
)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2256704990.py:5: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  .rolling("3d", on="timestamp")["geo_distance"]
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2256704990.py:4: Pandas4Warning: 'd' is deprecated and will be removed in a future version, please use 'D' instead.
  lambda x: x.sort_values("timestamp")
C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\2256704990.py:10: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to av

1130     4.250000
1792     5.190000
2603     5.306667
2685     5.672500
2901     5.952500
           ...   
41237    4.560000
42682    3.510000
45146    3.350000
46851    6.155000
49946    4.260000
Name: geo_distance_rolling, Length: 50000, dtype: float64

In [12]:
df["instant_risk"] = (
    df["amount_zscore"].abs() +
    df["device_change"] +
    (df["geo_distance"] / 10)
)


In [13]:
df["risk_momentum"] = (
    df.groupby("user_id")["instant_risk"]
    .diff()
)

df["risk_momentum"].fillna(0, inplace=True)


C:\Users\jadha\AppData\Local\Temp\ipykernel_16360\3757365766.py:6: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df["risk_momentum"].fillna(0, inplace=True)


1130     0.000000
1792     0.000000
2603     0.371602
2685    -0.820200
2901     0.252694
           ...   
41237    0.009262
42682   -0.197676
45146   -0.164943
46851    0.813539
49946   -0.471015
Name: risk_momentum, Length: 50000, dtype: float64

In [14]:
df["strategy_change"] = (
    df.groupby("user_id")["fraud_strategy_version"]
    .diff()
    .fillna(0)
    .abs()
)


In [15]:
feature_cols = [
    "user_id",
    "timestamp",
    "amount",
    "time_since_last_txn",
    "txn_count_1h",
    "amount_zscore",
    "device_change_rolling",
    "geo_distance_rolling",
    "risk_momentum",
    "strategy_change",
    "channel",
    "location",
    "is_fraud"
]


features_df = df[feature_cols].copy()
features_df.head()


,user_id,timestamp,amount,time_since_last_txn,txn_count_1h,amount_zscore,device_change_rolling,geo_distance_rolling,risk_momentum,strategy_change,channel,location,is_fraud
1130,1,2024-01-03 03:48:02,5.15,NaN,1.0,NaN,0.0,4.250000,NaN,0.0,card,rural,0
1792,1,2024-01-04 07:10:57,37.05,98575.0,1.0,0.677090,0.0,5.190000,NaN,0.0,netbanking,semi-urban,0
2603,1,2024-01-05 18:50:51,137.87,128394.0,1.0,1.107692,0.0,5.306667,0.371602,0.0,card,rural,0
2685,1,2024-01-05 22:04:41,47.32,11630.0,1.0,-0.164491,0.0,5.672500,-0.820200,0.0,netbanking,rural,0
2901,1,2024-01-06 06:48:59,93.75,31458.0,1.0,0.557185,0.0,5.952500,0.252694,0.0,card,semi-urban,0


In [16]:
features_df.to_csv(
    "../data/processed/features.csv",
    index=False
)

print("Feature dataset saved to data/processed/features.csv")


Feature dataset saved to data/processed/features.csv
